In [0]:
## import libraries 
from keras.layers import Dense,LSTM,Embedding,GRU,Input,Dropout,RepeatVector,Flatten
from keras.models import Sequential,Model
from keras.preprocessing.text import Tokenizer
import numpy as np
import keras.utils as KU
import traceback
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras import optimizers
import tensorflow as tf
import keras.losses as KL
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard

Using TensorFlow backend.


In [0]:
num_words=1000

In [0]:

##read dataset
with open('quora_duplicate_questions.tsv',encoding="utf-8") as f:
  data=f.read().split('\n')

In [0]:
corpus=[]
for i in data:
  line=i.split('\t')
  if(i!= " " and len(line)>5):
    is_duplicate=line[5]
    if(is_duplicate=='1'):
        corpus.append(i)

In [0]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()

In [0]:
## Not create input text as English and output text as Hindi
try:
  for line in corpus[:10000]:
      ##print(line)
      if(line !=""):
          text=line.split('\t')
          input_text=text[3]
          target_text=text[4]
          input_texts.append(input_text.lower())
          target_text='ssss '+target_text.lower()+' eeee'
          ##print(target_text)
          target_texts.append(target_text)
          for char in input_text:
              if char not in input_characters:
                  input_characters.add(char)
              
          for char in target_text:
              if char not in target_characters:
                  target_characters.add(char)
except Exception as e:
  print(e)
  traceback.print_exc()

In [0]:
max_sequence_len_input = max([len(x) for x in input_texts])
max_sequence_len_target = max([len(x) for x in target_texts])

input_tokenizer=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
input_tokenizer.fit_on_texts(input_texts)

target_tokenizer=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
target_tokenizer.fit_on_texts(target_texts)

input_vocab_size=len(input_tokenizer.word_index)+1
target_vocab_size=len(target_tokenizer.word_index)+1

In [0]:

## Now Generate the embeddings from Glove vector
embeddings_index={}
def Generate_Embeddings():
    f=open('../content/glove.6B.100d.txt') 
    for line in f:
        #print(line)
        values=line.split()
        word=values[0]
        coefs=np.array(values[1:],dtype="float32")
        embeddings_index[word]=coefs
    f.close()

## Now Generate Metrix of embedding as per our data
embedding_matrix = np.zeros((input_vocab_size, 100))
def Generate_Embeddings_Matrix():
    for word, i in input_tokenizer.word_index.items():
        #print(word,":::",i)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [0]:
Generate_Embeddings()

In [0]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [0]:

encoder_input_data=encode_sequences(input_tokenizer,max_sequence_len_input,input_texts)
decoder_input_data=encode_sequences(target_tokenizer,max_sequence_len_target,target_texts)
decoder_input_data_final=decoder_input_data[:,:-1]
decoder_target_data=decoder_input_data[:,1:]

In [0]:
## create an encoder archetecture
encoder_input=Input(shape=(None,))
encoder_embeddings=Embedding(input_vocab_size, 100, input_length=max_sequence_len_input-1, mask_zero=True,trainable=False)
encoder_gru_1=GRU(512,name="encoder_gru_1",return_sequences=True)
encoder_gru_2=GRU(512,name="encoder_gru_2",return_sequences=True)
encoder_gru_3=GRU(512,name="encoder_gru_3",return_sequences=False)


W0712 10:12:01.270819 139780771780480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0712 10:12:01.307568 139780771780480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [0]:
def Create_Encoder():
  #Here we will use functional api of keras
  net=encoder_input
  net=encoder_embeddings(net)
  ## not need to connect GRU units
  net=encoder_gru_1(net)
  net=encoder_gru_2(net)
  net=encoder_gru_3(net)
  
  return net


In [0]:
##Now create decoder architecture
decoder_initial_state=Input(shape=(512,),name="decoder_initial_state")
decoder_input=Input(shape=(None,),name="decoder_input")
decoder_embeddings=Embedding(target_vocab_size, 100, input_length=max_sequence_len_target-1,trainable=False)
decoder_gru_1=GRU(512,name="decoder_gru_1",return_sequences=True)
decoder_gru_2=GRU(512,name="decoder_gru_2",return_sequences=True)
decoder_gru_3=GRU(512,name="decoder_gru_3",return_sequences=True)
decoder_dense=Dense(target_vocab_size,activation="linear")

In [0]:
def Create_Decoder(initial_state):
  #Here we will use functional api of keras
  net=decoder_input
  net=decoder_embeddings(net)
  ## not need to connect GRU units
  net=decoder_gru_1(net,initial_state=initial_state)
  net=decoder_gru_2(net,initial_state=initial_state)
  net=decoder_gru_3(net,initial_state=initial_state)
  net=decoder_dense(net)
  
  return net

In [0]:
##Create a Training Model
encoder_output=Create_Encoder()
decoder_output=Create_Decoder(encoder_output)
model_train=Model(inputs=[encoder_input,decoder_input],outputs=[decoder_output])

W0712 10:12:08.180418 139780771780480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0712 10:12:08.754595 139780771780480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
## create an Encoder Model
model_encoder=Model(inputs=[encoder_input],outputs=[encoder_output])

In [0]:
## create a decoder model
decoder_output=Create_Decoder(decoder_initial_state)
model_decoder=Model(inputs=[decoder_input,decoder_initial_state],outputs=[decoder_output])

In [0]:
optimizer=optimizers.RMSprop(lr=1e-3)

In [0]:
decoder_target=tf.placeholder(dtype='int32',shape=(None,None))

In [0]:
model_train.compile(optimizer=optimizer, loss=KL.sparse_categorical_crossentropy,metrics=['acc'])
model_train.summary()


W0712 10:12:31.899124 139780771780480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0712 10:12:31.911848 139780771780480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3341: The name tf.log is deprecated. Please use tf.math.log instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 147, 100)     234700      input_1[0][0]                    
__________________________________________________________________________________________________
encoder_gru_1 (GRU)             (None, 147, 512)     941568      embedding_1[0][0]                
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
encoder_gr

In [0]:
path_checkpoint='21_checkpoint.keras'
callback_checkpoint=ModelCheckpoint(filepath=path_checkpoint,monitor='val_loss',verbose=1, save_best_only=True, save_weights_only=True)
callback_earlystopping=EarlyStopping(monitor='val_loss',verbose=1,patience=3)
callback_tensorboard=TensorBoard(log_dir='../content/21_logs/',histogram_freq=0,write_graph=False)
callbacks=[callback_checkpoint,callback_earlystopping,callback_tensorboard]

In [0]:
x_data={
    'encoder-input':encoder_input_data,
    'decoder-input':decoder_input_data
}
y_data={
    'decoder_output':decoder_target_data
}

In [0]:
model_train.fit([encoder_input_data, decoder_input_data], decoder_target_data.reshape(10000, 269,1),
          batch_size=640,
          epochs=20,
          validation_split=0.3,callbacks=callbacks)

ValueError: ignored

In [0]:
model_train.save('ModelNew.h5')

In [0]:
decoder_target_data.shape

(10000, 269)